In [14]:
#导入multiprocessing中的dummy方法中的Pool类
from multiprocessing.dummy import Pool
def calc_num2(num):
    return num*num
pool=Pool(3) #初始化一个有3个线程的线程池
num_1=[x for x in range(10)]
result=pool.map(calc_num2,num_1)  #第一个参数是执行的函数名，第二个参数是导入函数的具体参数
print('计算0-9的平方分别为：{}'.format(result))


计算0-9的平方分别为：[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [5]:
import requests #抓取源码模块
import time  #计时模块
from multiprocessing.dummy import Pool
def catching(html):
    html_1=requests.get(html)
    return html_1
start=time.time()
for i in range(100):
    catching('http://baidu.com')
end=time.time()
print('单线程循环访问100次百度首页，耗时：{}'.format(end-start))

start=time.time()
u_list=[]
for i in range(100):
    u_list.append('http://baidu.com')
pool=Pool(5)
pool.map(catching,u_list)
end=time.time()
print('5线程访问100次百度首页，耗时：{}'.format(end-start))


单线程循环访问100次百度首页，耗时：19.068340301513672
5线程访问100次百度首页，耗时：5.096128463745117


In [ ]:
#join函数,格式：‘sep’.join(list)
#0s.path.join('路径1'，,'路径2')
#输出路径为路径1\路径2，自动用分隔符拼接好

In [61]:

import requests
import re
import os
import time
from multiprocessing.dummy import Pool
start=time.time()
start_html='https://www.kanunu8.com/book3/6879/'
def get_html(url):  #网页源代码抓取模块
    html = requests.get(url)
    return html.content.decode('gbk') #这个网页需要使用gbk方式解码才能让中文正常显示

def get_toc(html):  #抓取每个章节的网址模块
    toc_block=[]
    toc_list=[]
    toc_html=[]
    toc_block=re.findall('正文(.*?)</tbody>',html,re.S)
    toc_html=re.findall('href="(.*?)"',toc_block[0],re.S)  #注意有toc_block[0]，toc_html是列表
    for ur1 in toc_html:
        toc_list.append(start_html+ur1)
    return toc_list

def get_article(html): #抓取章节标题和内容模块
    chapter_name=re.search('size="4">(.*?)<',html,re.S).group(1)
    text_block=re.search('<p>(.*?)</p>',html,re.S).group(1)
    text_block=text_block.replace('<br />','')
    return chapter_name,text_block
def save(chapter,article):
    os.makedirs('动物农场',exist_ok=True) #创建一个文件夹
    with open(os.path.join('动物农场',chapter+'.txt'),'w',encoding='utf-8') as f: #写文件,目录用os模块中的path.join方法自动生成路径
        f.write(article)                                                         #动物农场/chapter.txt
def every_article(url):   #输入每个章节网址，抓取每个章节然后保存
    article_html = get_html(url)
    chapter_name, article_text = get_article(article_html)
    save(chapter_name, article_text)
if __name__ == '__main__':
    toc_html = get_html(start_html)
    toc_list = get_toc(toc_html)
    pool = Pool(4) #线程池为4
    pool.map(every_article, toc_list)
end=time.time()
print('爬取整个小说耗时：{}\n'.format(end-start))
print('小说可去代码所在目录处查看')

        
        
        
        

爬取整个小说耗时：19.29182457923889

小说可去代码所在目录处查看


In [ ]:
#   XPath语句格式：//div[@class="useful"/ul/li/text()]

In [1]:
#XPath语句格式，书P486
import lxml.html  #调用lxml中的html方法
html_1='''
<!DOCTYPE html>
<html>
<head lang="en">
      <meta charset="UTF-8">
      <title></title>
</head>
<body>
      <div id="test-1">需要的内容1</div>
      <div id="test-2">需要的内容2</div>
      <div id="testfault">需要的内容3</div>
      <div id="useless">这是我不需要的内容</div>
</body>
</html>
'''
selector=lxml.html.fromstring(html_1)
content=selector.xpath('//div[starts-with(@id,"test")]/text()') #以相同字符串开头
for i in content:
    print(i)




需要的内容1
需要的内容2
需要的内容3


In [35]:
import lxml.html  #调用lxml中的html方法
html_2='''
<!DOCTYPE html>
<html>
<head lang="en">
      <meta charset="UTF-8">
      <title></title>
</head>
<body>
      <div id="abc-key">需要的内容1</div>
      <div id="123-key-000">需要的内容2</div>
      <div id="haha-key">需要的内容3</div>
      <div id="useless">这是我不需要的内容</div>
</body>
</html>
'''
selector=lxml.html.fromstring(html_2)
content_1=selector.xpath('//div[contains(@id,"-key")]/text()')  #属性值包含相同字段
for i in content_1:
    print(i)

需要的内容1
需要的内容2
需要的内容3


In [47]:
import lxml.html  #调用lxml中的html方法
html_3='''
<html>
      <head lang="en">
      <meta charset="UTF-8">
      <title></title>
 </head>
 <body>
      <div class="useful">
        <ul>
         <li class="info">需要的内容1</li>
         <li class="info">需要的内容2</li>
         <li class="info">需要的内容3</li>
        </ul>
      </div>
      <div class="useless">
        <ul>
         <li class="info">垃圾1</li> 
         <li class="info">垃圾2</li> 
        </ul>
      </div> 
 </body>
 </html>
'''
selector=lxml.html.fromstring(html_3)
content_2=selector.xpath('//div[@class="useful"]')
content_3=content_2[0].xpath('ul/li/text()') #先抓大再抓小，子xpath开头不需要添斜线
print(content_3)                             #[0]含义是先取div结点内内容，再xpath抓取里面内容




['需要的内容1', '需要的内容2', '需要的内容3']


In [51]:
html3='''
<! DOCTYPE html>
    <html>
    <head lang="en">
        <meta charset="UTF-8">
        <title>标题</title>
    </head>
    <body>
        <div id="test3">
          我左青龙，

     <span id="tiger">
              右白虎，
              <ul>上朱雀，
                  <li>下玄武。</li>
              </ul>
              老牛在当中，
          </span>
          龙头在胸口。
        </div>
    </body>
    </html>
'''
selector=lxml.html.fromstring(html3)
p=selector.xpath('//div[@id="test3"]') #只获取这个结点，不获取里面的东西
content_4=p[0].xpath('string(.)')
print(content_4)


          我左青龙，

     
              右白虎，
              上朱雀，
                  下玄武。
              
              老牛在当中，
          
          龙头在胸口。
        


In [22]:
#利用Google浏览器爬贴吧数据
import requests
import lxml.html
ul=requests.get("https://tieba.baidu.com/f?kw=%E6%B5%81%E6%B7%8C%E7%9A%84%E7%BE%8E%E5%A5%BD%E6%97%B6%E5%85%89").content.decode()
selector=lxml.html.fromstring(ul)
p=selector.xpath('//li[@class=" j_thread_list clearfix"]/div[@class="t_con cleafix"]/div[@class="col2_right j_threadlist_li_right "]/div[@class="threadlist_lz clearfix"]/div[@class="threadlist_title pull_left j_th_tit "]/a/text()')

for i in p:
    print(i)



真的受不了郑爽演技
有多少人为了一直都能做朋友而迟迟不敢迈出那一步......
哭不出来用纸遮
顾森湘的父母是不是傻？
有人看了吗？？结局是什么。。。？？？求告知
大结局还算圆满，他们还是在一起了
郝甜和杜度怎么那么傻！有没有人和我一样超级讨厌这俩
感觉越来越讨厌电视剧的易瑶了，演的真的很讨厌，反而觉得唐小米
我要给编剧寄红包！！！！！
求遥铭续写，好想看啊
齐铭的一声老婆，甜晕了😀😀😀😘😘
睡前再次感叹一句，流淌的美好时光真的是一部好剧#流淌的美好时
求问每次郝湉出场的背景纯音乐钢琴曲是什么歌？
流淌的美好时光
流淌的美好时光续写{祝易瑶齐铭}
流淌的没好时光
流淌的美好时光1-51集整理好了 有需要的评论自取哦～
求流淌的美好时光全集资源
流淌的美好时光1-51集整理好了  有需要的评论自取哦～
流淌的美好时光1-51集 大家有需要的评论自取呀～
流淌的美好时光
话说唐小米什么时候才会被揭穿？齐铭什么时候才知道唐小米对易遥
流淌的美好时光  好看  真有意思。
剧本写作技巧？剧本怎么写？
这部电视最让我意难平的不是唐小米做不做牢，也不是顾森湘的死而
如果不喜欢可以别看这部剧啊，觉得演技不好的你们可以拒绝，可以
看完电视剧的吧有，我想问一下，骆荀到底和谁在一起了？谢谢，万
**自有天收
以小的亏损，赌大的利益。
有没有哪个大神有大结局资源，红包看，不想等了
如果天宇和郑爽真的在一起该有多好呀……
我恋爱了🙈
有没有哪位大神知道齐铭当时唱的英文歌是哪首？有知道的能不能告
看了第一集，郑爽和马天宇说话口齿不清，听得很难受，郑爽更严重
顾森湘死了，齐铭和易遥肯定都会自责，他们还能坦然地在一起吗？
电视剧里易遥怀孕了吗？
看预告，感觉要还原原著了，心疼我易遥
顾森湘是自杀了吗？还是？
唐小米真坏
今天看了一个觉得很有意思的电视剧，好像是叫流淌的美好时光，当
是该分手冷静一下了——不过，不是齐铭的错
电视剧里面齐铭是不是没喜欢过顾森湘啊
对于齐名父亲这种明知道小三是贪自己的钱为什么还非要离婚和小三
不一样的“湘湘”，同样的雅君（湘湘扮演者）喜欢湘湘或者雅君的
郑爽的演技真是太尴尬了，她妈死了那里
武汉工程科技学院欢迎你，爱了爱了
奇迹看似遥不可及，但只有我们付出努力，那看似不可能发生的奇迹
小爽和天宇哥好友夫妻相，哈哈


In [21]:
import requests
import csv
import lxml.html
url=requests.get('https://www.damai.cn/projectlist.do').content.decode()
selector=lxml.html.fromstring(url)
item_list=selector.xpath('//div[2]/div[2]/div[1]/div[3]/div[1]/div/div[2]')
item_dict_list=[]
for item in item_list:
    show_name=item.xpath('div[@class="items__txt"]/div[@class="items__txt__title"]/a/text()')
    show_place=item.xpath('div[@class="items__txt"]/div[@class="items__txt__time"]/a[@class="items__txt__venue__icon"]/text()')
    show_time=item.xpath('div[@class="items__txt"]/div[@class="items__txt__time"]/a[@class="items__txt__time__icon"]/text()')
    show_price=item.xpath('div[@class="items__txt"]/div[@class="items__txt__price"]/span/text()')

    item_dict = {'show_name': show_name[0] if show_name else '',
                 'show_description': show_description[0] if show_description else '',
                 'show_time': show_time[0].strip() if show_time else '',
                 'show_place': show_place[0] if show_place else '',
                 'show_price': show_price[0] if show_price else ''}
    item_dict_list.append(item_dict)
print(item_list)
with open('result.csv','w',encoding='utf-8') as f:
    writer=csv.DictWriter(f,fieldnames=['show name','show place','show time','show price'])
    writer.writeheader()
    writer.writerows(item_dict_list)


[]


In [8]:
#sudo service mongod start
#sudo service mongod restart
#sudo service mongod stop

from pymongo import MongoClient
client=MongoClient()
#第一种方式初始化数据库
database=client.Chapter6  #创建数据库名称
collection=database.spider #创建数据表名称

#第二种方式
database=client['Chapter6']
collention=database['spider']

#方式二常用，处理批量数据时使用
database_name_list=['db1','db2','db3','db4']
for each_db in database_name_list:
    db=client[each_db]
    collection=db.test
    


In [16]:
#先在终端输入命令启动mongo:sudo service mongod start
#ps -ef | grep mongod   检查是否启动成功
#sudo service mongod stop
from pymongo import MongoClient
client_1=MongoClient()
database=client_1['ku_1']
collection=database['spider']
data={'id':112233,'name':'znn','age':21,'salary':'888'}
collection.insert(data) #插入字典数据

/home/tzy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


ObjectId('5d43fe12238d5c7be8a2621c')

In [19]:
more_data=[{'id':11,'name':'znn1','age':21,'salary':'888'},{'id':22,'name':'tzy1','age':21,'salary':'000'},{'id':33,'name':'txy1','age':22,'salary':'111'}]
database_1=client_1['ku_2']
collection_1=database_1['spider']
collection_1.insert(more_data)

/home/tzy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  after removing the cwd from sys.path.


[ObjectId('5d44390c238d5c7be8a26223'),
 ObjectId('5d44390c238d5c7be8a26224'),
 ObjectId('5d44390c238d5c7be8a26225')]

In [44]:
#普通查找
#find(筛选条件，选择返回哪些key)
content=[x for x in collection_1.find({'age':21})] #返回年龄都为21的数据
content

[{'_id': ObjectId('5d4429fe238d5c7be8a26220'),
  'id': 11,
  'name': 'znn',
  'age': 21,
  'salary': '888'},
 {'_id': ObjectId('5d4429fe238d5c7be8a26221'),
  'id': 22,
  'name': 'tzy',
  'age': 21,
  'salary': '000'},
 {'_id': ObjectId('5d4429fe238d5c7be8a26222'),
  'id': 33,
  'name': 'txy',
  'age': 21,
  'salary': '111'},
 {'_id': ObjectId('5d44390c238d5c7be8a26223'),
  'id': 11,
  'name': 'znn1',
  'age': 21,
  'salary': '888'},
 {'_id': ObjectId('5d44390c238d5c7be8a26224'),
  'id': 22,
  'name': 'tzy1',
  'age': 21,
  'salary': '000'}]

In [45]:
content=[x for x in collection_1.find({'salary':'111'},{'_id':0,'name':1,'age':1})] #第一个参数是返回年龄都为21的数据，第二个参数，0为不返回，1为返回该关键词
content                                                                           #不写出的字段默认不返回，_id比较特殊，必须指定0或1

[{'name': 'txy', 'age': 21}, {'name': 'txy1', 'age': 22}]

In [48]:
#逻辑查询
#    $gt 大于 $lt 小于 $gte 大于等于  $lte 小于等于  $eq 等于   $ ne 不等于
content=[x for x in collection_1.find({'age':{'$gt':20,'$lt':22}}).sort('salary',1)] #sort方法，1为升序，-1为降序
content
print('降序\t')
content=[x for x in collection_1.find({'age':{'$gt':20,'$lt':22}}).sort('salary',-1)] #sort方法，1为升序，-1为降序
content

降序	


[{'_id': ObjectId('5d4429fe238d5c7be8a26220'),
  'id': 11,
  'name': 'znn',
  'age': 21,
  'salary': '888'},
 {'_id': ObjectId('5d44390c238d5c7be8a26223'),
  'id': 11,
  'name': 'znn1',
  'age': 21,
  'salary': '888'},
 {'_id': ObjectId('5d4429fe238d5c7be8a26222'),
  'id': 33,
  'name': 'txy',
  'age': 21,
  'salary': '111'},
 {'_id': ObjectId('5d4429fe238d5c7be8a26221'),
  'id': 22,
  'name': 'tzy',
  'age': 21,
  'salary': '000'},
 {'_id': ObjectId('5d44390c238d5c7be8a26224'),
  'id': 22,
  'name': 'tzy1',
  'age': 21,
  'salary': '000'}]

In [51]:
# update_one({'要修改的关键词':  },{'$set':{'修改成的关键词':}}) 第一个参数可以用逻辑查询
# update_many({'要修改的关键词':  },{'$set':{'修改成的关键词':}}) 第一个参数可以用逻辑查询
collection_1.update_many({'age':21},{'$set':{'age':30}})  #21岁都修改为30岁
collection_1.delete_many({'age':22}) #删除
content=[x for x in collection_1.distinct('age')] #返回一个去重后的年龄的列表
content

[30, 22]

In [62]:
# 启动redis需要在终端输入redis-cli
#用redis—py操作redis
#本地redis
import redis
client=redis.StrictRedis()
#列表左侧添加数据用lpush,左侧用l，右侧用r
client.lpush('chapter_6',123)
client.llen('chapter_6') #算长度

[b'123', b'123', b'123', b'123', b'123', b'123', b'123', b'123']

In [63]:
client.lrange('chapter_6',0,15) #读取列表0-15数据
# lpop 从列表右侧读一个值，读取后删除该数据

[b'123',
 b'123',
 b'123',
 b'123',
 b'123',
 b'123',
 b'123',
 b'123',
 b'4',
 b'3',
 b'2',
 b'1',
 b'world',
 b'hello']

In [ ]:
#MongoDB与Redis的使用。其中，MongoDB主要用来存放爬虫爬到的各种需要持久化保存的数据，
#而Redis则用来存放各种中间数据。通过减少频繁读/写MongoDB，用Redis来弥补MongoDB的一些不足，可以显著提高爬虫的运行效率

In [ ]:
client.sadd('test_set','www.baidu.com')  #集合中添加一个网址
# spop是从集合中读取一个值，然后删去
# scard 查看集合的长度

In [83]:
import requests #爬虫1将每个章节的网址全部爬取到redis上
import redis
from lxml import html
client=redis.StrictRedis()
source=requests.get('http://dongyeguiwu.zuopinj.com/5525').content
selector=html.fromstring(source)
url_list=selector.xpath('//div[@class="book_list"]/ul/li/a/@href') # @href这样的方式可以提取出href后面的章节网址链接
for url in url_list:
    client.lpush('url_queue',url) 

In [84]:
import requests
import redis
import lxml.html
import pymongo
client=redis.StrictRedis()
connection=pymongo.MongoClient()
db=connection.chapter_6
handler=db.white

connection_list=[]
while client.llen('url_queue')>0:
    url=client.lpop('url_queue').decode() #读取redis中储存的每一章网址数据
    source=requests.get(url).content      #抓取每个章节源码
    selector=lxml.html.fromstring(source)
    chapter_name=selector.xpath('//div[@class="h1title"]/h1/text()')[0] #章节标题
    content=selector.xpath('//div[@id="htmlContent"]/p/text()')         #章节内容
    connection_list.append({'title':chapter_name,'content':'\n'.join(content)})
handler.insert_many(connection_list)  # insert_one  insert_many 用法不同
